# Lösung: Generierung der Exportformate für Lingo und Solr aus dem Bibsonomy-Suchergebnis

In [1]:
import json
import requests
from langdetect import detect

url = "https://www.bibsonomy.org/json/search/Bibliothek?items=1000&duplicates=merged"

result = requests.get(url)
result # Ausgabe des Ergebnisses - gibt Typ (Response) und HTTP-Statuscode (idealerweise 200) aus

<Response [200]>

In [2]:
data = result.json()
data # Ansicht der Ausgabe

{'types': {'Bookmark': {'pluralLabel': 'Bookmarks'},
  'Publication': {'pluralLabel': 'Publications'},
  'GoldStandardPublication': {'pluralLabel': 'GoldStandardPublications'},
  'GoldStandardBookmark': {'pluralLabel': 'GoldStandardBookmarks'},
  'Tag': {'pluralLabel': 'Tags'},
  'User': {'pluralLabel': 'Users'},
  'Sphere': {'pluralLabel': 'Spheres'}},
 'properties': {'count': {'valueType': 'number'},
  'date': {'valueType': 'date'},
  'changeDate': {'valueType': 'date'},
  'url': {'valueType': 'url'},
  'id': {'valueType': 'url'},
  'tags': {'valueType': 'item'},
  'user': {'valueType': 'item'}},
 'items': [{'type': 'Bookmark',
   'id': 'https://www.bibsonomy.org/url/8dccb0fb43c1baa049c04533af10c807/hotho',
   'tags': ['bibliothek', 'treffpunkt'],
   'intraHash': '8dccb0fb43c1baa049c04533af10c807',
   'label': 'Bibliothek: Treffpunkt Bibliothek',
   'user': 'hotho',
   'description': 'Treffpunkt Bibliothek 24. - 31. Oktober 2010',
   'date': '2010-07-18 21:30:25',
   'changeDate': '2

In [3]:
items = data['items']

publications = [] # neue leere Liste
for item in items:
    if item['type']=='Publication': # Bedingung für Aufnahme des Items in die neue Liste
        publications.append(item) # item an Liste anhängen

#mit list comprehension:
#publications = [item for item in items if item['type'] == 'Publication']

german_pubs = []
for pub in publications:
    label = pub['label']
    label = label.strip() # Leerzeichen an Anfang und Ende des Strings entfernen
    language = detect(label) # detect-Funktion auf String anwenden
    
    if language == 'de':
        german_pubs.append(pub)
        
#mit list comprehension:
#german_pubs = [item for item in publications if detect(item['label'].strip()) == 'de']

In [4]:
german_pubs[0] # ein Item der Ergebnisliste mal zum Prüfen ausgeben

{'type': 'Publication',
 'id': 'https://www.bibsonomy.org/bibtex/26134ca4f1047738d19be7aab9fa3191d/wdees',
 'tags': ['forschungsdaten', 'bibliothek', 'merged:wdees', 'oer'],
 'intraHash': '6134ca4f1047738d19be7aab9fa3191d',
 'interHash': '832c51690660dc4168a2906c9cae3e42',
 'label': 'Data EDUcation an der UDE – Eine OER für Bibliotheken',
 'user': 'wdees',
 'description': 'Data EDUcation an der UDE – Eine OER für Bibliotheken : Bibliothek Forschung und Praxis',
 'date': '2019-12-16 10:31:41',
 'changeDate': '2019-12-16 10:31:41',
 'count': 1,
 'pub-type': 'article',
 'journal': 'Bibliothek Forschung und Praxis',
 'year': '2019',
 'url': 'https://www.degruyter.com/view/j/bfup.2019.43.issue-3/bfp-2019-2078/bfp-2019-2078.xml',
 'author': ['Christina Kläre', 'Kim-Sarah Jung'],
 'authors': [{'first': 'Christina', 'last': 'Kläre'},
  {'first': 'Kim-Sarah', 'last': 'Jung'}],
 'volume': '43',
 'number': '3',
 'pages': '387--398',
 'issn': '03414183',
 'doi': '10.1515/bfp-2019-2078',
 'bibtexKe

## Generieren des Lingo-Exportformats

Das Format für Lingo sieht so aus:  

```
[id.]
text.

[id.]
text.

<...>
```

Die ID muss numerisch sein, daher ist es am einfachsten, diese für jeden Datensatz hochzuzählen.

Als Text verwenden wir die beiden textuellen Felder 'label' (=Titel), und, wenn vorhanden, 'abstract'.

In [5]:
# Einschub: String-Konkatenation
a = '['
b = 'hallo'
c = ']'

concatenated = a+b+c # mit + lassen sich Strings einfach zusammenfügen
print(concatenated)

[hallo]


In [6]:
with open('output/bibsonomy_to_lingo.txt', 'w', encoding="utf-8") as outfile:
    for index, data in enumerate(german_pubs): # enumerate gibt Tupel (index, item) zurück
        index_str = str(index) # zunächst die Zahl zum String umwandeln
        
        outfile.write('['+index_str+'.]\n') # Index-String mit den anderen Zeichenfolgen verbinden
        
        outfile.write(data['label']+'\n')
        
        if 'abstract' in data:
            abstract = data['abstract']
            abstract_clean = abstract.replace("\n\n", " ") # bereinigen des Abstract
            
            outfile.write(abstract_clean+'\n')
        outfile.write('\n')

## Generieren des Solr-Exportformats

Solr akzeptiert bereits JSON als Importformat, daher könnten wir auch beinahe den Datensatz wie er jetzt ist übergeben. Selbst eine id ist darin bereits vorhanden (es handelt sich um die eindeutige bibsonomy URL zum Datensatz).

Wir wollen allerdings die Daten vorher noch etwas bereinigen, insbesondere unnötige Felder weglassen und Felder mit schlechten Bezeichnungen umbenennen ('label' -> 'title').

In [7]:
for item in german_pubs:
    item['title'] = item.pop('label') # Key umbenennen
    item.pop('intraHash', None) # alternativ: löschen mit del
    #del item['intraHash']
    item.pop('interHash', None)
    item.pop('user', None)
    item.pop('dnbtitleid', None)
    item.pop('bibtexKey', None)    

In [8]:
#Ausgabe des Ergebnisses zum Prüfen
print(json.dumps(german_pubs, indent=4))

[
    {
        "type": "Publication",
        "id": "https://www.bibsonomy.org/bibtex/26134ca4f1047738d19be7aab9fa3191d/wdees",
        "tags": [
            "forschungsdaten",
            "bibliothek",
            "merged:wdees",
            "oer"
        ],
        "description": "Data EDUcation an der UDE \u2013 Eine OER f\u00fcr Bibliotheken : Bibliothek Forschung und Praxis",
        "date": "2019-12-16 10:31:41",
        "changeDate": "2019-12-16 10:31:41",
        "count": 1,
        "pub-type": "article",
        "journal": "Bibliothek Forschung und Praxis",
        "year": "2019",
        "url": "https://www.degruyter.com/view/j/bfup.2019.43.issue-3/bfp-2019-2078/bfp-2019-2078.xml",
        "author": [
            "Christina Kl\u00e4re",
            "Kim-Sarah Jung"
        ],
        "authors": [
            {
                "first": "Christina",
                "last": "Kl\u00e4re"
            },
            {
                "first": "Kim-Sarah",
                "last": "

In [9]:
with open('output/bibsonomy_to_solr.json', 'w', encoding="utf-8") as outfile:
    json.dump(german_pubs, outfile, indent=4)